<a href="https://colab.research.google.com/github/francoishcm/BackTesting/blob/master/Zipline_Reloaded_BacktestEngine_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🧰# **CODE TO INSTALL MODULES & PACKAGES**

In [ ]:
# Mount Google Drive 
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Install ta-lib v0.4.0
%%bash
wget http://prdownloads.sourceforge.net/ta-lib/ta-lib-0.4.0-src.tar.gz
tar -xzf ta-lib-0.4.0-src.tar.gz
cd ta-lib/
./configure
make
make install

In [ ]:
# Install zipline
%pip install zipline-reloaded

In [ ]:
# Install matplot library
!pip install matplotlib

In [ ]:
# Install data bundle 'Quandl'
!pip install quandl

In [ ]:
# Ingest Quandl using API key
!QUANDL_API_KEY=KUnssHvVERHb5XYu9C1- zipline ingest -b 'quandl'

In [ ]:
# Confirm existing bundle that were ingested
!zipline bundles

In [ ]:
# Install stockstats
!pip install stockstats

In [ ]:
# Install Pyfolio
!pip install git+https://github.com/quantopian/pyfolio

# 💽# **CODE TO SET AND CONFIRM CURRENT WORKING DIRECTORIES**

In [ ]:
import os 

# Set your working directory to a folder in your Google Drive. This way, if your notebook times out,
# your files will be saved in your Google Drive!

# the base Google Drive directory
root_dir = "/root/content/drive/"


In [ ]:
import os 

# Set your working directory to a folder in your Google Drive. This way, if your notebook times out,
# your files will be saved in your Google Drive!


# choose where you want your project files to be saved
project_folder = "MyDrive/Colab Notebooks/My Project Folder"


In [ ]:
def create_and_set_working_directory(project_folder):
  # check if your project folder exists. if not, it will be created.
  if os.path.isdir(root_dir + project_folder) == False:
    os.mkdir(root_dir + project_folder)
    print(root_dir + project_folder + ' did not exist but was created.')

  # change the OS to use your project folder as the working directory
  os.chdir(root_dir + project_folder)

  # create a test file to make sure it shows up in the right place
  !touch 'new_file_in_working_directory.txt'
  print('\nYour working directory was changed to ' + root_dir + project_folder + \
        "\n\nAn empty text file was created there. You can also run !pwd to confirm the current working directory." )

create_and_set_working_directory(project_folder)


Your working directory was changed to /root/content/drive/MyDrive/Colab Notebooks/My Project Folder

An empty text file was created there. You can also run !pwd to confirm the current working directory.


In [ ]:
# Confirm current working directory
!pwd

/root/content/drive/MyDrive/Colab Notebooks/My Project Folder


In [ ]:
import os
for f in os.listdir("/root/content/drive/MyDrive/Colab Notebooks/My Project Folder"):
	print(f)

new_file_in_working_directory.txt
random_futures
random_stocks.zip
.ipynb_checkpoints
stocks_at_random


# ⏳# **RUN CODE TO REGISTER & INGEST CUSTOM BUNDLES HERE**

In [77]:
!QUANDL_API_KEY=muQr9rZqHVUZsgSmF1WJ zipline ingest -b 'quandl'

[2022-04-07 20:39:16.678792] INFO: zipline.data.bundles.core: Ingesting quandl.
[2022-04-07 20:39:16.679024] INFO: zipline.data.bundles.quandl: Downloading WIKI metadata.
[2022-04-07 20:39:29.490873] INFO: zipline.data.bundles.quandl: Parsing raw data.
[2022-04-07 20:40:02.942743] INFO: zipline.data.bundles.quandl: Generating asset metadata.
Merging daily equity files:  [------------#-----------------------]  1731/usr/local/lib/python3.7/dist-packages/zipline/data/bcolz_daily_bars.py:366: UserWarning: Ignoring 1 values because they are out of bounds for uint32:             open  ...  split_ratio
2011-04-11  1.79  ...          1.0

[1 rows x 7 columns]
  winsorise_uint32(raw_data, invalid_data_behavior, "volume", *OHLC)
Merging daily equity files:  [####################################]      
[2022-04-07 20:42:23.739479] INFO: zipline.data.bundles.quandl: Parsing split data.
[2022-04-07 20:42:23.910891] INFO: zipline.data.bundles.quandl: Parsing dividend data.
[2022-04-07 20:42:25.60618

In [90]:
# Ingest custom bundle
!zipline ingest -b random

[2022-04-07 20:59:27.635913] INFO: zipline.data.bundles.core: Ingesting random.
 | APA: sid 0
 | BBY: sid 1
 | CMCSA: sid 2
 | CTL: sid 3
 | EL: sid 4
 | FAST: sid 5
 | GCO: sid 6
 | HPQ: sid 7
 | JCI: sid 8
 | JEC: sid 9
 | MCHP: sid 10
Loading custom pricing data:   [######################--------------]   61% | MSFT: sid 11
Loading custom pricing data:   [########################------------]   66% | PEP: sid 12
Loading custom pricing data:   [##########################----------]   72% | PTC: sid 13
Loading custom pricing data:   [############################--------]   77% | RAD: sid 14
Loading custom pricing data:   [##############################------]   83% | T: sid 15
Loading custom pricing data:   [################################----]   88% | TMK: sid 16
Loading custom pricing data:   [##################################--]   94% | VVI: sid 17
Loading custom pricing data:   [####################################]  100%
Merging daily equity files:  [###########################

In [89]:
!CSVDIR=/content/drive/MyDrive/Colab Notebooks/My Project Folder/random_stocks zipline ingest -b random

/bin/bash: Notebooks/My: No such file or directory


In [84]:
!zipline bundles

csvdir <no ingestions>
quandl 2022-04-07 20:39:16.179368
quantopian-quandl <no ingestions>
random 2022-04-07 20:36:33.049653


# **CLENOW CORE TREND MODEL CODE** *Do Not Delete*

In [91]:
%matplotlib inline

import zipline
from zipline.api import order_target_percent, symbol,  \
    set_commission, set_slippage, schedule_function, \
    date_rules, time_rules
from datetime import datetime
import pytz
import matplotlib.pyplot as plt
import pyfolio as pf
import pandas as pd
import numpy as np  
from scipy import stats  
from zipline.finance.commission import PerDollar
from zipline.finance.slippage import VolumeShareSlippage, FixedSlippage

"""
Model Settings
"""
intial_portfolio = 10000000
momentum_window = 125
minimum_momentum = 40
portfolio_size = 30
vola_window = 20

"""
Commission and Slippage Settings
"""
enable_commission = True
commission_pct = 0.001
enable_slippage = True 
slippage_volume_limit = 0.25
slippage_impact = 0.1

"""
Helper functions.
"""

def momentum_score(ts):
    """
    Input:  Price time series.
    Output: Annualized exponential regression slope, 
            multiplied by the R2
    """
    # Make a list of consecutive numbers
    x = np.arange(len(ts)) 
    # Get logs
    log_ts = np.log(ts) 
    # Calculate regression values
    slope, intercept, r_value, p_value, std_err = stats.linregress(x, log_ts)
    # Annualize percent
    annualized_slope = (np.power(np.exp(slope), 252) - 1) * 100
    #Adjust for fitness
    score = annualized_slope * (r_value ** 2)
    return score

def volatility(ts):
    return ts.pct_change().rolling(vola_window).std().iloc[-1]

def output_progress(context):
    """
    Output some performance numbers during backtest run
    This code just prints out the past month's performance
    so that we have something to look at while the backtest runs.
    """
    
    # Get today's date
    today = zipline.api.get_datetime().date()
    
    # Calculate percent difference since last month
    perf_pct = (context.portfolio.portfolio_value / context.last_month) - 1
    
    # Print performance, format as percent with two decimals.
    print("{} - Last Month Result: {:.2%}".format(today, perf_pct))
    
    # Remember today's portfolio value for next month's calculation
    context.last_month = context.portfolio.portfolio_value

"""
Initialization and trading logic
"""
def initialize(context):
   
    # Set commission and slippage.
    if enable_commission:
        comm_model = PerDollar(cost=commission_pct)
    else:
        comm_model = PerDollar(cost=0.0)
    set_commission(comm_model)
    
    if enable_slippage:
        slippage_model=VolumeShareSlippage(volume_limit=slippage_volume_limit, price_impact=slippage_impact)
    else:
        slippage_model=FixedSlippage(spread=0.0)   
    set_slippage(slippage_model)    
    
    # Used only for progress output.
    context.last_month = intial_portfolio
    
    # Fetch and store index membership
    context.index_members = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/My Project Folder/index members/sp500.csv', index_col=0, parse_dates=[0])
    
    #Schedule rebalance monthly.
    schedule_function(
        func=rebalance,
        date_rule=date_rules.month_start(),
        time_rule=time_rules.market_open()
    )
    
def rebalance(context, data):
    # Write some progress output during the backtest
    output_progress(context)
    
    # Ok, let's find which stocks can be traded today.

    # First, get today's date
    today = zipline.api.get_datetime()
    
    # Second, get the index makeup for all days prior to today.
    all_prior = context.index_members.loc[context.index_members.index < today]
    
    # Now let's snag the first column of the last, i.e. latest, entry.
    latest_day = all_prior.iloc[-1,0]
    
    # Split the text string with tickers into a list
    list_of_tickers = latest_day.split(',')
    
    # Finally, get the Zipline symbols for the tickers
    todays_universe = [symbol(ticker) for ticker in list_of_tickers]
    
    # There's your daily universe. But we could of course have done this in one go.
    """       
    
    # This line below does the same thing,
    # using the same logic to fetch today's stocks.
    
    todays_universe = [
        symbol(ticker) for ticker in 
        context.index_members.loc[context.index_members.index < today].iloc[-1,0].split(',')
    ]
    """    
    
    # Get historical data
    hist = data.history(todays_universe, "close", momentum_window, "1d")

    # Make momentum ranking table
    ranking_table = hist.apply(momentum_score).sort_values(ascending=False)  
    
    """
    Sell Logic
    
    First we check if any existing position should be sold.
    * Sell if stock is no longer part of index.
    * Sell if stock has too low momentum value.
    """
    kept_positions = list(context.portfolio.positions.keys())
    for security in context.portfolio.positions:
        if (security not in todays_universe):
            order_target_percent(security, 0.0)
            kept_positions.remove(security)          
        elif ranking_table[security] < minimum_momentum:
            order_target_percent(security, 0.0)
            kept_positions.remove(security)         
      
    
    """
    Stock Selection Logic
    
    Check how many stocks we are keeping from last month.
    Fill from top of ranking list, until we reach the
    desired total number of portfolio holdings.
    """
    replacement_stocks = portfolio_size - len(kept_positions)
    buy_list = ranking_table.loc[
        ~ranking_table.index.isin(kept_positions)][:replacement_stocks]
    
    new_portfolio = pd.concat(
        (buy_list, 
         ranking_table.loc[ranking_table.index.isin(kept_positions)])
    )
    

    """
    Calculate inverse volatility for stocks, 
    and make target position weights.
    """
    vola_table = hist[new_portfolio.index].apply(volatility)
    inv_vola_table = 1 / vola_table 
    sum_inv_vola = np.sum(inv_vola_table)         
    vola_target_weights = inv_vola_table / sum_inv_vola
    
    for security, rank in new_portfolio.iteritems():
        weight = vola_target_weights[security]
        if security in kept_positions:
            order_target_percent(security, weight)
        else:
            if ranking_table[security] > minimum_momentum:
                order_target_percent(security, weight)

def analyze(context, perf):
    
    perf['max'] = perf.portfolio_value.cummax()
    perf['dd'] = (perf.portfolio_value / perf['max']) - 1
    maxdd = perf['dd'].min()
    
    ann_ret = (np.power((perf.portfolio_value.iloc[-1] / perf.portfolio_value.iloc[0]),(252 / len(perf)))) - 1
    
    print("Annualized Return: {:.2%} Max Drawdown: {:.2%}".format(ann_ret, maxdd))

    return   
        
start = datetime(1997, 1, 1, 8, 15, 12, 0, pytz.UTC)
end = datetime(2018, 12, 31, 8, 15, 12, 0, pytz.UTC)
perf = zipline.run_algorithm(
    start=start, end=end, 
    initialize=initialize, 
    analyze=analyze, 
    capital_base=intial_portfolio,  
    data_frequency = 'daily', 
    bundle= 'random' ) 



UnknownBundle: ignored

# 💻# **RUN BACKTEST CODE HERE**

In [83]:
# Import a few libraries we need
import zipline
from zipline import run_algorithm
from zipline.api import order_target_percent, symbol, schedule_function, date_rules, time_rules
from datetime import datetime
import pytz
import pyfolio as pf
import pandas as pd

def initialize(context):
  # Which stock to trade
  dji = ['APA','BBY','CMCSA','CTL','EL','FAST','GCO','HPQ','JCI','JEC','MCHP','MSFT']
  # Make symbol list from tickers
  context.universe = [symbol(s) for s in dji]
  # History window
  context.history_window = 20
  # Size of our portfolio
  context.stocks_to_hold = 10
  # Schedule the daily trading routine for once per month
  schedule_function(handle_data, date_rules.month_start(), time_rules.market_close())
def month_perf(ts):
  perf = (ts[-1] / ts[0]) - 1
  return perf
def handle_data(context, data):
  # Get history for all the stocks.
  hist = data.history(context.universe, "close", context.history_window, "1d")
  # This creates a table of percent returns, in order.
  perf_table = hist.apply(month_perf).sort_values(ascending=False)
  # Make buy list of the top N stocks
  buy_list = perf_table[:context.stocks_to_hold]
  # The rest will not be held.
  the_rest = perf_table[context.stocks_to_hold:]
  # Place target buy orders for top N stocks.
  for stock, perf in buy_list.iteritems():
    stock_weight = 1 / context.stocks_to_hold
    # Place order
    if data.can_trade(stock):
      order_target_percent(stock, stock_weight)
    # Make sure we are flat the rest.
  for stock, perf in the_rest.iteritems():
    # Place order
    if data.can_trade(stock):
      order_target_percent(stock, 0.0)
def analyze(context, perf):
  # Use PyFolio to generate a performance report
  returns, positions, transactions = pf.utils.extract_rets_pos_txn_from_zipline(perf)

  pf.create_returns_tear_sheet(returns, benchmark_rets=None)
# Set start and end date
start_date = pd.Timestamp('2003-1-1', tz='utc')
end_date = pd.Timestamp('2017-12-31', tz='utc')
# Fire off the backtest
reresult = run_algorithm(
    start=start_date,
    end=end_date,
    initialize=initialize,
    analyze=analyze,
    handle_data=handle_data,
    capital_base=10000,
    data_frequency = 'daily',
    bundle= 'random')

UnknownBundle: ignored

# 📈# **SAVE PERFORMANCE RESULTS TO CSV**

In [ ]:
#@title
# Store performance results (equity curve) as csv file on Drive
perf.portfolio_value.to_csv('model_performance.csv')